In [ ]:
!pip install xlrd

In [ ]:
import os
from google.colab import drive

drive.mount('/gdrive', force_remount=True)
root_dir = '/gdrive/My Drive/tests/dexma/'

In [3]:
# Muting python warnings
import warnings
warnings.filterwarnings('ignore')

# Numpy for array manipulation
import numpy as np

# Pandas for working with dataframes and series
import pandas as pd

# Sklearn packages for metrics, data transformation, feature selection and forecast models
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, LassoCV

From here it's possible to create new features calculated only on train set data

In [67]:
def prediction_absolute_prop_error(y, y_pred):
    return np.mean(np.abs((y - y_pred) / y))

In [107]:
train = pd.read_csv('train_X.csv')
val = pd.read_csv('val_X.csv')

In [108]:
val.head()

,Unnamed: 0,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id,year,month
0,60,1,12,10,2017-01-01,12.712153,11.105362,0.0,0.0,0.0,0.001951,0.0,0.0,61,2017,1
1,61,1,12,10,2017-02-01,13.395923,11.789671,0.0,0.0,0.0,-0.010809,0.0,0.0,62,2017,2
2,62,1,12,10,2017-03-01,13.148651,11.512138,0.0,0.0,0.0,0.000000,0.0,0.0,63,2017,3
3,63,1,12,10,2017-04-01,13.236274,11.552861,0.0,0.0,0.0,0.000000,0.0,0.0,64,2017,4
4,64,1,12,10,2017-05-01,14.359559,12.558873,0.0,0.0,0.0,-0.071319,0.0,0.0,65,2017,5


In [109]:
train.drop(['date', 'Unnamed: 0', 'sales1'] , axis=1, inplace=True)
val.drop(['date', 'Unnamed: 0', 'sales1'], axis=1, inplace=True)

In [110]:
train.rename({'sales2': 'y'}, axis = 1, inplace=True)
val.rename({'sales2': 'y'}, axis = 1, inplace=True)

In [111]:
y_train = train.y
X_train = train.drop('y', axis=1)
y_val = val.y
X_val = val.drop('y', axis=1)

In [112]:
scaler = StandardScaler()
model = LassoCV()

In [113]:
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [114]:
result = model.fit(X_train, y_train)

In [115]:
y_pred = model.predict(X_val)

In [116]:
print('Test set mean absolute proportion error: {:.2f}'.format(prediction_absolute_prop_error(y_pred, y_val)))

Test set mean absolute proportion error: 10.59


In [77]:
y_pred

array([  48.71382517,   49.20337481,   49.02633922, ..., 3722.44918281,
       3934.71782674, 3799.86095937])

In [91]:
val_pred = val.copy()
val_pred.drop('Unnamed: 0', axis=1, inplace=True)
val_pred['y_hat'] = y_pred

In [92]:
val_pred.head()

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id,year,month,y_hat
0,1,12,10,2017-01-01,12.712153,11.105362,0.0,0.0,0.0,0.001951,0.0,0.0,61,2017,1,48.713825
1,1,12,10,2017-02-01,13.395923,11.789671,0.0,0.0,0.0,-0.010809,0.0,0.0,62,2017,2,49.203375
2,1,12,10,2017-03-01,13.148651,11.512138,0.0,0.0,0.0,0.000000,0.0,0.0,63,2017,3,49.026339
3,1,12,10,2017-04-01,13.236274,11.552861,0.0,0.0,0.0,0.000000,0.0,0.0,64,2017,4,49.089073
4,1,12,10,2017-05-01,14.359559,12.558873,0.0,0.0,0.0,-0.071319,0.0,0.0,65,2017,5,49.893297


In [93]:
val_pred.to_csv('pred_lasso_pablo_1.csv')